In [5]:
### check versions of key python libraries
#scipy 
import scipy
print('scipy: %s' % scipy.__version__)
#numpy
import numpy
print('numpy: %s' % numpy.__version__)
# matplotlib
import matplotlib
print('matplotlib: %s' % matplotlib.__version__)
# pandas
import pandas
print('pandas: %s' % pandas.__version__)
# statsmodels
import statsmodels
print('statsmodels: %s' % statsmodels.__version__)
# scikit-learn
import sklearn
print('sklearn: %s' % sklearn.__version__)


scipy: 1.7.2
numpy: 1.21.4
matplotlib: 3.5.0
pandas: 1.3.4
statsmodels: 0.13.0
sklearn: 1.0.1
